# Brief sketch of Julia's secret sauce

Notebook from [HelloJulia.jl](https://github.com/ablaom/HelloJulia.jl)

Julia's *secret sauce:*

- **Just-in-time compilation**
- **Multiple dispatch**
- **Abstract types**

If your just copying code into Julia's REPL, then you can ignore the
next line:

In [1]:
using InteractiveUtils

## Just-in-time compilation

Here's how we define a new function in Julia:

In [2]:
add(x, y) = x + y

add (generic function with 1 method)

Let's see how long it takes to add two numbers:

In [3]:
@time @eval add(3, 5)

  0.006770 seconds (1.75 k allocations: 127.982 KiB, 67.88% compilation time)


8

Slow!! Why? Because Julia is a *compiled* language and does not
compile new code until it knows the type of arguments you want to
use. (The use of the macro `@eval` helps us to include this
compilation time in the total measurement, since `@time` is designed
to cleverly exclude it in recent Julia versions.)

Let's try again *with the same type* of argument:

In [4]:
@time @eval add(4, 7)

  0.000102 seconds (41 allocations: 2.516 KiB)


11

Fast!!! Why? Because Julia caches the compiled code and the types
are the same. We can even inpsect an annotated version of this
compiled code:

In [5]:
@code_llvm add(4, 7)

;  @ string:1 within `add'
define i64 @julia_add_884(i64 signext %0, i64 signext %1) {
top:
; ┌ @ int.jl:87 within `+'
   %2 = add i64 %1, %0
; └
  ret i64 %2
}


This code is indistinguishable from analogous C code (if using the `clang` compiler).

Let's try vectors:

In [6]:
x = rand(3)
y = rand(3)

3-element Vector{Float64}:
 0.03477884794902297
 0.41987273682512827
 0.5425939657541741

In [7]:
@time @eval add(x, y)

  0.100438 seconds (268.62 k allocations: 15.490 MiB, 99.83% compilation time)


3-element Vector{Float64}:
 0.3754108828525524
 1.2084209377250128
 1.479993893449279

Slow :-(

In [8]:
@time @eval add(y, x)

  0.000150 seconds (42 allocations: 2.625 KiB)


3-element Vector{Float64}:
 0.3754108828525524
 1.2084209377250128
 1.479993893449279

Fast :-).

Just-in-time compilation exists in other languages (eg, Java).

## Multiple dispatch

In [9]:
y = [1 2; 3 4]

2×2 Matrix{Int64}:
 1  2
 3  4

In [10]:
typeof(y)

Matrix{Int64} (alias for Array{Int64, 2})

Julia doesn't know how to apply `+` to a scalar and a
matrix. Uncomment the following line (by removing the "#" symbol) to
see the error thrown:

In [11]:
# add(4, y)

So we add a more specialized version of our function (called a
*method*) to handle this case:

In [12]:
add(x::Int64, y::Matrix{Int64}) = x .+ y

add (generic function with 2 methods)

Here we are using the built-in broadcasted version of `+` which adds
the scalar `x` to each element of `y`. Now this works:

In [13]:
add(4, y)

2×2 Matrix{Int64}:
 5  6
 7  8

This is essentially what multiple dispatch is about. We use *all*
the arguments of a function to determine what specific method to
call. In a traditional object oriented language methods are owned by
objects (data structures) and we see syntax like `x.add(y)` which is
*single* dispatch on `x`.

Multiple dispatch is not used in any widely used languages. Dylan is
the most well-known example.

If you are coming from a traditional object oriented language like
Python, then you're used to thinking of objects "owning" methods. In
Julia *functions*, not objects, own *methods*:

In [14]:
methods(add)

# 2 methods for generic function "add":
[1] add(x::Int64, y::Matrix{Int64}) in Main.##257 at string:1
[2] add(x, y) in Main.##257 at string:1

Or, stated differently, there is less conflation of *structure* and
*behaviour* in Julia!

But, we're not out of the woods yet. Uncomment to see a new error
thrown:

In [15]:
# add(4.0, y)

Oh dear. Do we need to write a special method for every kind of
scalar and matrix???!

No, because abstract types come to the rescue...

## Abstract types

Everything in Julia has a type:

In [16]:
typeof(1 + 2im)

Complex{Int64}

In [17]:
typeof(rand(2,3))

Matrix{Float64} (alias for Array{Float64, 2})

These are examples of *concrete* types. But concrete types have
*supertypes*, which are *abstract*:

In [18]:
supertype(Int64)

Signed

In [19]:
supertype(Signed)

Integer

In [20]:
supertype(Integer)

Real

And we can travel in the other direction:

In [21]:
subtypes(Real)

4-element Vector{Any}:
 AbstractFloat
 AbstractIrrational
 Integer
 Rational

In [22]:
4 isa Real

true

In [23]:
Bool <: Integer

true

In [24]:
String <: Integer

false

Now we can solve our problem: How to extend our `add` function to
arbitrary scalars and matrices:

In [25]:
add(x::Real, y::Matrix) = x .+ y

add (generic function with 3 methods)

In [26]:
add(4.0, rand(Bool, 2, 3))

2×3 Matrix{Float64}:
 5.0  4.0  5.0
 4.0  4.0  4.0

Note that abstract types have no instances. The only "information"
in an abstract type is what its supertype and subtypes
are. Collectively, abstract types and concrete types constitute a
tree structure, with the concrete types as leaves. This structure
exists to *organize* the concrete types in a way that facilitates
extension of functionality. This tree is not static, but can be
extended by the programmer.

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*